In [3]:
import numpy as np
def isprime(x):
  if x == 2 or x == 3: return True
  if x < 2 or x%2 == 0: return False
  if x < 9: return True
  if x%3 == 0: return False
  a = int(x**0.5)
  b = 5
  while b <= a:
    if x%b == 0: return False
    if x%(b+2) == 0: return False
    b=b+6
  return True
def factor(x):
  a = int(x**0.5)
  for p in range(2,a+1):
    if x%p == 0: return p
  return 1

def dump(*ls): 
  for s in ls: print (s, "=", eval(s))
rng = np.random.default_rng()

# Everybody: Run the code below to generate your public-private key pair

In [12]:
pmin = 103
pmax = 1003
p = q = e = n = t = 0
while p==0 or not isprime(p):         p = rng.integers(pmin,pmax+1).item()
while q==0 or not isprime(q) or q==p: q = rng.integers(pmin,pmax+1).item()
n = p*q
t = (p-1)*(q-1)
while e==0 or np.gcd(e,t)!=1:         e = rng.integers(2,t+1).item()
d = pow (e, -1, t)
print (f'Generating first random prime factor:  p = {p}')
print (f'Generating second random prime factor: q = {q}')
print (f'Calculating public modulus:            n = p * q       = {n}')
print (f'Calculating private modulus:           t = (p-1)*(q-1) = {t}')
print (f'Generating public encryption key:      e = {e}     (must be coprime with t)')
print (f'Generating private decryption key:     d = {d}')
print (f'\nWrite the following on a piece of paper and show it to everyone:\n\n\tMY PUBLIC KEY IS (e,n) = {(e,n)}')
print (f'\nDo not share your private key!')

Generating first random prime factor:  p = 773
Generating second random prime factor: q = 271
Calculating public modulus:            n = p * q       = 209483
Calculating private modulus:           t = (p-1)*(q-1) = 208440
Generating public encryption key:      e = 43651     (must be coprime with t)
Generating private decryption key:     d = 69211

Write the following on a piece of paper and show it to everyone:

	MY PUBLIC KEY IS (e,n) = (43651, 209483)

Do not share your private key!


# Alice: Run the code below

In [17]:
eBob,nBob = eval(input ("Enter Bob's public key as two integers separated by a comma: (e,n) = "))
m = eval(input (f"Enter the message you want to send (must be an integer smaller than {nBob}): m = "))
c = pow (m, eBob, nBob)
#dump ('n','e','c')
print (f'\nWrite the following on a piece of paper and pass it to Bob:\n\n\tHEY BOB: c = {c}')

Enter Bob's public key as two integers separated by a comma: (e,n) =  43651,209483
Enter the message you want to send (must be an integer smaller than 209483): m =  12345



Write the following on a piece of paper and pass it to Bob:

	HEY BOB: c = 81557


# Bob: Run the code below

In [21]:
c = eval(input ("Enter the ciphertext you received from Alice: c = "))
m = pow (c, d, n)
print (f'Decrypting the message using your private key: m = pow(c,d,n) = {c} ** {d} % {n} = {m}')
print (f'\nThe decrypted message is:\n\n\tm = {m}')

Enter the ciphertext you received from Alice: c =  81557


Decrypting the message using your private key: m = pow(c,d,n) = 81557**69211%209483 = 12345

The decrypted message is:

	m = 12345


# Eve: Run the code below

In [28]:
eBob,nBob = eval(input ("Enter Bob's public key as two integers separated by a comma: (e,n) = "))
c = eval(input ("Enter the ciphertext that Alice sent to Bob, which you intercepted: c = "))
print (f'\nNow you need to find the value of m such that pow(m,e,n) = c.\nThat is, find m such that\n\n\tm ** {e} % {n} = {c}.')

Enter Bob's public key as two integers separated by a comma: (e,n) =  43651,209483
Enter the ciphertext that Alice sent to Bob, which you intercepted: c =  81557



Now you need to find the value of m such that pow(m,e,n) = c.
That is, find m such that

tm ** 43651 % 209483 = 81557.


In [29]:
for m in range(nBob):
  if pow(m,eBob,nBob)==c:
    print (f'Decryption by brute force: m = {m}')

Decryption by brute force: m = 12345


# Evan: Run the code below

In [1]:
eBob,nBob = eval(input ("Enter Bob's public key as two integers separated by a comma: (e,n) = "))
print (f"To crack Bob's public key to obtain his private key, we need to factor n = {nBob}.")

Enter Bob's public key as two integers separated by a comma: (e,n) =  43651,209483


To crack Bob's public key to obtain his private key, we need to factor n = 209483.


In [5]:
p = factor (nBob)
q = nBob//p
print (f'Factorize by brute force: {nBob} = {p} * {q} = n = p*q where p={p} and q={q}')
tBob = (p-1)*(q-1)
dBob = pow (eBob, -1, tBob)
dump ('p','q','tBob','dBob')
print (f"Now we know that Bob's private key is dBob = {dBob}.")

Factorize by brute force: 209483 = 271 * 773 = n = p*q where p=271 and q=773
p = 271
q = 773
tBob = 208440
dBob = 69211
Now we know that Bob's private key is dBob = 69211.


In [11]:
c = eval(input ("Enter the ciphertext that Alice sent to Bob, which you intercepted: c = "))
m = pow (c, dBob, nBob)
print (f"Decrypting the message using Bob's private key, which you obtained by cracking: m = pow(c,d,n) = {c} ** {dBob} % {nBob} = {m}")
print (f'\nThe decrypted message is:\n\n\tm = {m}')

Enter the ciphertext that Alice sent to Bob, which you intercepted: c =  81557


Decrypting the message using Bob's private key, which you obtained by cracking: m = pow(c,d,n) = 81557 ** 69211 % 209483 = 12345

The decrypted message is:

	m = 12345
